In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import sunpy.map
from aiapy.calibrate.prep import register as aiaprep  # sunpy.instr.aia.aiaprep was moved to aiapy
from sunpy.net import Fido, attrs as a
from astropy import units as u

import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
# manual inputs
t = a.Time('2014-04-18T18:19:04', '2014-04-18T19:15:54')

folder = '/Users/delbe/OneDrive - FHNW/aia_data/2014_12_04T18_19_04'
# folder_iris = "../../../../cr_obs3/20140418_123338_3820259153/Si IV 1400"
p1 = [1420,1353]  # x,y of p1 (begin of slit in AIA coordinates)
p2 = [1422,1255]  # x,y of p2 (end of slit in AIA coordinates)

In [3]:
# search for aia sources
result = Fido.search(t, a.Instrument("aia"), a.Wavelength(304*u.angstrom), a.Sample(1*u.second))

In [4]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
object,object,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str57
2014-04-18 18:19:07.000,2014-04-18 18:19:08.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:19:19.000,2014-04-18 18:19:20.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:19:31.000,2014-04-18 18:19:32.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:19:43.000,2014-04-18 18:19:44.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:19:55.000,2014-04-18 18:19:56.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:20:07.000,2014-04-18 18:20:08.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:20:19.000,2014-04-18 18:20:20.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2014-04-18 18:20:31.000,2014-04-18 18:20:32.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"


In [ ]:
# download files, skip if already downloaded
# Fido.fetch(result, path=folder)

In [6]:
# get downloaded files
files = glob.glob(folder + '/*')
for file in files:print(file)

In [ ]:
# This should be modified in such a way that it reads the input from the excel sheet and calculates all the images. 

gradient = (p2[1]-p1[1])/(p2[0]-p1[0])
origin = p1[1] - gradient * p1[0]

# x = np.linspace(p1[0], p2[0], 200)  # 200 = slit-size in pixels?
# y = (gradient*x + origin)
# coords = (np.column_stack((x.astype(int),y.astype(int))))

# alternative way to get coordinates - assuming slit is always (almost) vertical
y = np.arange(min(p1[1], p2[1]),max(p1[1], p2[1])+1)
x = (y - origin) / gradient
coords = (np.column_stack((x.round().astype(int),y.astype(int))))
print(coords)
final_image = []

for i in range(0,len(files)-1):
    
# for i in range(0,100):
    
    if i%10 == 0:
        print(i, end=',')
        
    
    try:         
        aia1 = sunpy.map.Map(files[i])
        
    except: 
        
        continue
        
    aia = aiaprep(aia1)
    
    zi = np.flipud(aia.data.T)[coords[:,1], coords[:,0]]
   
    
    if len(final_image) == 0: 
        final_image = zi 
    else:
        final_image = np.vstack([final_image, data])

        
# plt.figure(figsize=[15,10]) # on this image, we should see the downflows.
# plt.imshow(final_image.transpose()) # Please adapt the axes for the correct annotation (y-solar coordinates and x-time)